# 🪐 Projeto - Pipeline Bronze , Silver e Gold no Databricks

🥉 A camada Bronze consiste em **armazenar os arquivos brutos exatamente como recebidos**, sem qualquer tratamento, limpeza ou parsing.  

🥈 A camada Silver consiste no tratamento e padronização dos arquivos, para facilitar a sua leitura.

🥇 A camada Gold consiste na visualização dos dados através de gráficos e os respectivos dados ja tratado na camada silver.

- O objetivo aqui é **preservar a integridade dos dados originais**, sem alterá-los.  
- No Databricks, utilizamos o método `text()` para armazenar os dados como linhas de texto puro, mantendo os acentos e caracteres como estão no arquivo de origem.

In [0]:
# 🥉 Bronze - Exibir o conteúdo bruto do arquivo, sem tratamento

# Leitura de cada arquivo como texto (linha a linha, sem parsear CSV)
df_atendimentos_bronze = spark.read.text("dbfs:/FileStore/tables/atendimentos.csv")
df_cirurgias_bronze = spark.read.text("dbfs:/FileStore/tables/cirurgias_sus.csv")
df_exames_bronze = spark.read.text("dbfs:/FileStore/tables/exames.csv")

# Mostrar uma amostra das linhas brutas (cada linha como uma string)
print("Atendimentos - Bronze (dados brutos):")
df_atendimentos_bronze.show(5, truncate=False)

print("Cirurgias - Bronze (dados brutos):")
df_cirurgias_bronze.show(5, truncate=False)

print("Exames - Bronze (dados brutos):")
df_exames_bronze.show(5, truncate=False)

# Copiar os arquivos originais para a camada Bronze, como foram recebidos
dbutils.fs.cp("dbfs:/FileStore/tables/atendimentos.csv", "dbfs:/mnt/bronze/atendimentos.csv")
dbutils.fs.cp("dbfs:/FileStore/tables/cirurgias_sus.csv", "dbfs:/mnt/bronze/cirurgias_sus.csv")
dbutils.fs.cp("dbfs:/FileStore/tables/exames.csv", "dbfs:/mnt/bronze/exames.csv")


Atendimentos - Bronze (dados brutos):
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                       |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|atendimento_id,paciente_id,nome_paciente,doenca,localidade,especialidade,local_atendimento,data_atendimento,data_consulta,medico                                                            |
|60cd2245-b9c8-48cb-a48f-927efaad3509,8b34bd8e-e008-4e80-820f-9f55950a521b,Ana Julia Sousa,deserunt,Farias de Peixoto,repellendus,Alves Vasconcelos - EI,2021-06-02,2023-10-02,Francisco Lima|
|cc5603

In [0]:
# 🥈 Silver - leitura formatada dos CSVs
path_atendimentos_bronze = "dbfs:/mnt/bronze/atendimentos.csv"
path_cirurgias_bronze = "dbfs:/mnt/bronze/cirurgias_sus.csv"
path_exames_bronze = "dbfs:/mnt/bronze/exames.csv"

# Leitura formatada
df_atendimentos = (
    spark.read
        .format("csv")
        .option("header", "true")
        .option("sep", ",")
        .option("encoding", "UTF-8")
        .load(path_atendimentos_bronze)
)

df_cirurgias = (
    spark.read
        .format("csv")
        .option("header", "true")
        .option("sep", ",")
        .option("encoding", "UTF-8")
        .load(path_cirurgias_bronze)
)

df_exames = (
    spark.read
        .format("csv")
        .option("header", "true")
        .option("sep", ",")
        .option("encoding", "UTF-8")
        .load(path_exames_bronze)
)

# Visualização já formatada
print("Atendimentos - Silver:")
display(df_atendimentos.limit(100))
print("Cirurgias - Silver:")
display(df_cirurgias.limit(100))
print("Exames - Silver:")
display(df_exames.limit(100))

# Salvar em Parquet
df_atendimentos.write.mode("overwrite").format("parquet").save("/mnt/silver/atendimentos")
df_cirurgias.write.mode("overwrite").format("parquet").save("/mnt/silver/cirurgias")
df_exames.write.mode("overwrite").format("parquet").save("/mnt/silver/exames")

Atendimentos - Silver:


atendimento_id,paciente_id,nome_paciente,doenca,localidade,especialidade,local_atendimento,data_atendimento,data_consulta,medico
60cd2245-b9c8-48cb-a48f-927efaad3509,8b34bd8e-e008-4e80-820f-9f55950a521b,Ana Julia Sousa,deserunt,Farias de Peixoto,repellendus,Alves Vasconcelos - EI,2021-06-02,2023-10-02,Francisco Lima
cc56033d-9d04-4aae-a00b-4062d983b1b0,5230cac3-ca7a-4664-a7eb-ddf8ac566ff9,Srta. Maria Flor Casa Grande,laudantium,Sá das Flores,quasi,Porto S/A,2022-09-30,2023-05-10,Dom Peixoto
e6b37369-6cff-4ae8-ac0c-e26bd9d64f09,a397f8be-faee-4367-a076-63128eaaaf11,Liam Moura,laborum,Pastor,ex,Costa - EI,2020-11-26,2022-06-28,Léo Fernandes
320f82d0-1832-456e-a3fa-93ed95d87821,c86209f6-ef67-4635-9c88-3ea84a4eae0b,Carolina Guerra,molestiae,Porto,inventore,da Costa,2022-06-18,2022-11-13,Stella Martins
70da2315-da79-4c45-9067-170f2d1d4e8f,0d972679-1452-420d-ac57-ab57cc28cf77,Benjamim Costela,laboriosam,da Costa Paulista,ullam,Machado - EI,2022-07-10,2024-01-14,Eduardo Oliveira
62a8fd77-85a6-4a18-994e-55ff9b8ddb11,dde3531c-5ed3-4509-bead-8f9fba487a02,Benjamin Nogueira,sequi,Ramos da Serra,a,Pinto,2020-12-28,2023-07-08,Carolina Azevedo
103c6bae-c036-490b-ba3a-c1e1105fe5c7,4a4eaced-332c-4730-bde9-22233dfe0277,Dr. Nathan Teixeira,optio,da Mota da Prata,ut,Correia,2023-06-11,2023-02-23,Valentina da Mata
a5ff6485-8173-42ff-b8fd-a39a27e12871,72b9a4c1-8946-4589-964c-ad4ccd7657cc,Sophie Gomes,laboriosam,Peixoto de Goiás,eligendi,da Cunha S.A.,2024-06-24,2022-07-15,Rodrigo Fonseca
94622a23-d9b8-4e46-80ba-3cbc6d0ad364,5d71c509-66c5-44b9-b75b-2f5e1d894edc,Bruna da Mata,voluptates,Martins,laudantium,Siqueira e Filhos,2024-04-07,2022-12-13,Luiz Gustavo Rios
3178c253-427e-44cb-823b-7d6c7b7137f0,b24de0ea-b39b-4ac5-a5b1-e0cfdd1146bb,Gael Henrique Almeida,animi,Mendonça,tenetur,Viana,2020-07-29,2021-08-16,Aylla Melo


Cirurgias - Silver:


nome_paciente,data_cirurgia,cod_paciente,tipo_cirurgia,doenca,hospital,tipo_procedimento,nome_medico,resultado_cirurgia
Igor Lopes,2023-11-28,892256,Cirurgia torácica,Fratura óssea,Hospital São Paulo,Procedimento não invasivo,Rodrigo da Mota,Falha
João Miguel Ramos,2022-01-08,925794,Cirurgia ortopédica,Fratura óssea,Hospital Albert Einstein,Procedimento invasivo,Ana Lívia das Neves,Falha
Anna Liz Costa,2022-12-09,663278,Cirurgia plástica,Queimadura,Hospital Sírio-Libanês,Procedimento invasivo,Pedro da Mata,Complicações
Mirella Marques,2022-08-29,920507,Cirurgia plástica,Doença cardíaca,Hospital Albert Einstein,Procedimento invasivo,Sr. Pedro Henrique Fernandes,Sucesso
João Miguel da Mota,2023-08-03,494736,Cirurgia plástica,Doença pulmonar,Hospital Albert Einstein,Procedimento não invasivo,Benjamim Machado,Complicações
Sra. Raquel Azevedo,2024-10-23,640912,Cirurgia neurológica,Fratura óssea,Hospital Albert Einstein,Procedimento invasivo,Sr. Thales Dias,Falha
Maysa Nascimento,2024-03-08,866365,Cirurgia torácica,Queimadura,Hospital Sírio-Libanês,Procedimento não invasivo,Sra. Carolina Vargas,Complicações
Leandro Caldeira,2021-01-09,117949,Cirurgia neurológica,Doença pulmonar,Hospital das Clínicas,Procedimento não invasivo,Arthur Miguel Dias,Falha
João Pedro Novais,2022-05-25,367734,Cirurgia cardíaca,Doença pulmonar,Hospital São Paulo,Procedimento invasivo,Allana da Mota,Complicações
Fernando Novaes,2024-02-02,616787,Cirurgia plástica,Doença pulmonar,Hospital São Paulo,Procedimento não invasivo,Heloisa Costa,Falha


Exames - Silver:


exame_id,paciente_id,nome_paciente,nome_exame,data_realizacao,solicitante,localidade,especialidade_solicitante
545c41de-d297-41b4-a429-84fd052cb8bb,dcc3dd51-60a2-47cf-8a56-4de21f6af200,Pedro Lucas Abreu,veniam,2024-03-21,Nicole Almeida,Guerra,necessitatibus
5e808b6d-df45-4d75-9ed7-ceb322127915,058fe562-c221-4210-821f-5a48b7ced8d4,Raquel Pacheco,dolor,2020-07-18,Isabelly Fernandes,Rios,suscipit
994e5797-88dc-485a-af62-bfe1407ba1c7,3f5756f9-d00c-4248-bd91-8c8b6edfa290,Dr. Caio Silveira,nam,2023-03-20,Ísis Montenegro,Jesus,odit
23384a8b-d852-4880-8487-9c4412cc2d6f,4d2bcb72-6251-4e08-9f06-ebac80402304,João Vitor Fernandes,voluptatem,2024-09-02,Bruno Guerra,Moraes,possimus
bb364ec8-de4a-4674-b98b-6c34e244e8b5,b8ef1a3c-20d2-4672-958f-9d64a7ff2c07,Luigi Nogueira,delectus,2022-09-06,Raquel Pacheco,Fonseca,sed
32c593bf-7919-4808-8a8d-b94333074b0c,b2350800-fd4f-49b8-b97b-bb0fbed80c64,Marina das Neves,dignissimos,2024-01-29,Sra. Isabella Peixoto,Cirino de Lopes,reprehenderit
4d817b7f-bb95-4ca1-a6eb-ca8d48187c29,361ad41d-d355-4e27-84d2-6d619e6bbeb1,João Felipe Viana,labore,2024-04-12,Júlia da Costa,Moura de Cunha,eveniet
3c66a4e4-9454-4421-bc7b-8470ddd9740a,b3763d53-2add-4ac0-af9d-70848e848df1,Hellena Cirino,explicabo,2025-04-23,Heitor Guerra,Macedo do Campo,voluptatem
b969a170-5785-44e1-845e-88427386abbe,1b69fadc-568e-49af-80ef-b36d4fb52d8c,Melina da Rosa,recusandae,2025-01-16,Sr. Yago Gonçalves,Vasconcelos Alegre,vel
419c519d-6439-4720-ac80-d1f32eabfc56,ef7b7fb2-1ce2-4f3c-8bc3-ba0508adb82b,Sr. Lorenzo Araújo,saepe,2022-06-20,Camila Cavalcanti,Pimenta,dolore


In [0]:
# 🥇 Gold - Preparação para Visualização dos dados de 'atendimentos'

# IMPORTA AS BIBLIOETAS NECESSARIAS
from pyspark.sql.functions import year, month, count, col, to_date, regexp_replace, trim, when, lit

#  Carrega os dados da camada Silver (Parquet)
print("Carregando dados de atendimentos da camada Silver (Parquet)...")
df_atendimentos_silver = spark.read.format("parquet").load("/mnt/silver/atendimentos")
print("CARREGADO COM SUCESSO!")

# Garente que todos os nomes das colunas esteja certo.
print("\nEsquema do DataFrame de Atendimentos (Silver) para referência:")
df_atendimentos_silver.printSchema()

#Transformações e Agregações para a Camada Gold 
print("\nRealizando transformações e agregações para a camada Gold de atendimentos...")

# Limpeza e Agregação por 'local_atendimento'
PADROES_PARA_REMOVER_LOCAL = "(?i)(S/A| - EI|\\.{3}|Rua|Av\\.|Trav\\.|Avenida|Praca|Praça|Rodovia|Rod)" # <-- AJUSTE ESTA EXPRESSÃO REGULAR

# Aplicando a limpeza diretamente na coluna 'local_atendimento'
df_atendimentos_gold = df_atendimentos_silver.withColumn(
    "local_atendimento", 
    regexp_replace(
        col("local_atendimento"),
        PADROES_PARA_REMOVER_LOCAL,
        ""
    )
)

df_atendimentos_gold = df_atendimentos_gold.withColumn(
    "local_atendimento",
    trim(regexp_replace(col("local_atendimento"), "\\.$", "")) 
)
#  FINALIZANDO A LIMPEZA NA COLUNA ORIGINAL
df_atendimentos_gold = df_atendimentos_gold.withColumn(
    "local_atendimento",
    when(col("local_atendimento").isNull() | (trim(col("local_atendimento")) == ""), lit("Nao Informado"))
    .otherwise(col("local_atendimento"))
)

# Agrupa e conta usando a coluna 'local_atendimento' já limpa
df_atendimentos_gold_por_local = df_atendimentos_gold \
    .groupBy("local_atendimento") \
    .agg(count("*").alias("TotalAtendimentos")) \
    .orderBy(col("TotalAtendimentos").desc())


# Agregação por 'data_atendimento' (Mes/Ano) 
NOME_COLUNA_DATA_ATENDIMENTO = "data_atendimento" 
FORMATO_DATA_ATENDIMENTO = "yyyy-MM-dd" 

df_atendimentos_com_data_atendimento_formatada = df_atendimentos_silver.withColumn(
    "data_atendimento_formatada", to_date(col(NOME_COLUNA_DATA_ATENDIMENTO), FORMATO_DATA_ATENDIMENTO)
)

df_atendimentos_gold_por_data_atendimento = df_atendimentos_com_data_atendimento_formatada \
    .filter(col("data_atendimento_formatada").isNotNull()) \
    .groupBy(year("data_atendimento_formatada").alias("Ano"), \
             month("data_atendimento_formatada").alias("Mes")) \
    .agg(count("*").alias("TotalAtendimentos")) \
    .orderBy("Ano", "Mes")

# Agregação por 'medico'

NOME_COLUNA_MEDICO = "medico"

df_atendimentos_gold_por_medico = df_atendimentos_silver \
    .groupBy(col(NOME_COLUNA_MEDICO).alias("NomeMedico")) \
    .agg(count("*").alias("TotalAtendimentos")) \
    .orderBy(col("TotalAtendimentos").desc())

print("Agregações para gráficos de atendimentos concluídas.")

# Salva os dados agregados para o grafico na Camada Gold 
print("\nSalvando DataFrames agregados de atendimentos na camada Gold (Parquet)...")
df_atendimentos_gold_por_local.write.mode("overwrite").format("parquet").save("/mnt/gold/atendimentos_por_local")
df_atendimentos_gold_por_data_atendimento.write.mode("overwrite").format("parquet").save("/mnt/gold/atendimentos_por_data_atendimento")
df_atendimentos_gold_por_medico.write.mode("overwrite").format("parquet").save("/mnt/gold/atendimentos_por_medico")
print("Salvos com sucesso.")


#  Cria visualizaçoes temporárias para os gráficos
print("\nCriando temporárias a partir dos DataFrames da camada Gold...")
df_atendimentos_gold_por_local.createOrReplaceTempView("gold_atendimentos_por_local")
df_atendimentos_gold_por_data_atendimento.createOrReplaceTempView("gold_atendimentos_por_data_atendimento")
df_atendimentos_gold_por_medico.createOrReplaceTempView("gold_atendimentos_por_medico")
print("CRIADAS: 'gold_atendimentos_por_local', 'gold_atendimentos_por_data_atendimento', 'gold_atendimentos_por_medico'.")

Carregando dados de atendimentos da camada Silver (Parquet)...
CARREGADO COM SUCESSO!

Esquema do DataFrame de Atendimentos (Silver) para referência:
root
 |-- atendimento_id: string (nullable = true)
 |-- paciente_id: string (nullable = true)
 |-- nome_paciente: string (nullable = true)
 |-- doenca: string (nullable = true)
 |-- localidade: string (nullable = true)
 |-- especialidade: string (nullable = true)
 |-- local_atendimento: string (nullable = true)
 |-- data_atendimento: string (nullable = true)
 |-- data_consulta: string (nullable = true)
 |-- medico: string (nullable = true)


Realizando transformações e agregações para a camada Gold de atendimentos...
Agregações para gráficos de atendimentos concluídas.

Salvando DataFrames agregados de atendimentos na camada Gold (Parquet)...
Salvos com sucesso.

Criando temporárias a partir dos DataFrames da camada Gold...
CRIADAS: 'gold_atendimentos_por_local', 'gold_atendimentos_por_data_atendimento', 'gold_atendimentos_por_medico'.


In [0]:
%sql
SELECT local_atendimento, TotalAtendimentos
FROM gold_atendimentos_por_local
ORDER BY TotalAtendimentos DESC
LIMIT 20 

local_atendimento,TotalAtendimentos
Siqueira,653
Guerra,645
Machado,636
Moraes,626
Câmara,622
Leão,619
Rezende,619
Ferreira,614
Garcia,609
Alves,608


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT Ano, Mes, TotalAtendimentos
FROM gold_atendimentos_por_data_atendimento
ORDER BY Ano, Mes

Ano,Mes,TotalAtendimentos
2020,5,662
2020,6,1676
2020,7,1724
2020,8,1709
2020,9,1614
2020,10,1657
2020,11,1689
2020,12,1756
2021,1,1627
2021,2,1510


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT NomeMedico, TotalAtendimentos
FROM gold_atendimentos_por_medico
ORDER BY TotalAtendimentos DESC
LIMIT 10

NomeMedico,TotalAtendimentos
Otávio Barros,11
Ana Laura Andrade,11
Fernando Albuquerque,11
Anthony Sousa,10
Nicolas Nunes,10
Levi Correia,10
Luiz Miguel Almeida,10
Marcelo Cavalcanti,10
Larissa Nunes,10
Luiz Henrique Nascimento,10


Databricks visualization. Run in Databricks to view.